# Neural Network Smartphone Activity Detector

In this activity, you will train a neural network to use smartphone data to predict the activity of the user. 

This dataset has already been separated into input features and target activities. Additional information on the dataset can be found here. 

http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions

### Data Pre-Processing

Prepare the data for the neural network. This includes splitting the data into a training and testing dataset, Scaling the data, and encoding the categorical target values

In [80]:
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [81]:
# Read the input features into `X`
X = pd.read_csv(Path("./Resources/features.csv"), header=None)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,...,-0.012236,-0.314848,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718
1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,...,0.202804,-0.603199,-0.860677,0.053477,-0.007435,-0.732626,0.703511,-0.845092,0.180261,-0.047436
2,0.039978,-0.005153,-0.022651,-0.995482,-0.977314,-0.984760,-0.996415,-0.975835,-0.985973,-0.798477,...,0.440079,-0.404427,-0.761847,-0.118559,0.177899,0.100699,0.808529,-0.849230,0.180610,-0.042271
3,0.039785,-0.011809,-0.028916,-0.996194,-0.988569,-0.993256,-0.996994,-0.988526,-0.993135,-0.798477,...,0.430891,-0.138373,-0.491604,-0.036788,-0.012892,0.640011,-0.485366,-0.848947,0.181907,-0.040826
4,0.038758,-0.002289,-0.023863,-0.998241,-0.986774,-0.993115,-0.998216,-0.986479,-0.993825,-0.801982,...,0.137735,-0.366214,-0.702490,0.123320,0.122542,0.693578,-0.615971,-0.848164,0.185124,-0.037080


In [82]:
# Read the target values into `y`
y = pd.read_csv(Path("./Resources/target.csv"))
y.head()

,activity
0,standing
1,standing
2,standing
3,standing
4,standing


In [83]:
y.activity.value_counts()

standing              1423
laying                1413
sitting               1293
walking               1226
walking_upstairs      1073
walking_downstairs     987
stand_to_lie            90
sit_to_lie              75
lie_to_sit              60
lie_to_stand            57
stand_to_sit            47
sit_to_stand            23
Name: activity, dtype: int64

In [84]:
# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [85]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

In [86]:
# Scale the training and testing input features using StandardScaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [87]:
# Apply One-hot encoding to the target labels
enc = OneHotEncoder()

class_values = y["activity"].values.reshape(-1, 1)
enc.fit(class_values)

OneHotEncoder()

In [88]:
enc.categories_

[array(['laying', 'lie_to_sit', 'lie_to_stand', 'sit_to_lie',
        'sit_to_stand', 'sitting', 'stand_to_lie', 'stand_to_sit',
        'standing', 'walking', 'walking_downstairs', 'walking_upstairs'],
       dtype=object)]

In [89]:
type(enc.categories_)

list

In [90]:
# Encode cateogories
class_encoded = enc.transform(class_values).toarray()
class_encoded_df = pd.DataFrame(class_encoded, columns=enc.categories_)
class_encoded_df.head()

,laying,lie_to_sit,lie_to_stand,sit_to_lie,sit_to_stand,sitting,stand_to_lie,stand_to_sit,standing,walking,walking_downstairs,walking_upstairs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [91]:
class_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

# Build a Deep Neural Network

In [92]:
# Create a sequential model
model = Sequential()

In [93]:
# Add the first layer where the input dimensions are the 561 columns of the training data
number_inputs = 561
number_hidden_nodes = 6
model.add(Dense(units=number_hidden_nodes, activation="relu", input_dim=number_inputs))

In [94]:
# The output layer has 12 columns that are one-hot encoded
y_train.activity.value_counts()
number_outputs = 12

In [95]:
# Add output layer using 12 output nodes. 
# HINT: Use `softmax` as the activation 
number_classes = 1

model.add(Dense(units=number_outputs, activation="softmax"))

In [96]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [97]:
# Print the model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 6)                 3372      
_________________________________________________________________
dense_13 (Dense)             (None, 12)                84        
Total params: 3,456
Trainable params: 3,456
Non-trainable params: 0
_________________________________________________________________


In [99]:
# Use the training data to fit (train) the model
# @NOTE: Experiment with the number of training epochs to find the minimum iterations required to achieve a good accuracy
model_fit = model.fit(X_train_scaled, class_encoded, epochs=100)

ValueError: Data cardinality is ambiguous:
  x sizes: 5825
  y sizes: 7767
Make sure all arrays contain the same number of samples.

# Evaluate the Model

In [ ]:
# Evaluate the model using the testing data
# YOUR CODE HERE!

In [ ]:
# Make predictions
predicted = model.predict(X_test_scaled)
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.activity.values,
    "Predicted": predicted
})
results.head(10)

In [ ]:
# Print the Classification Report
from sklearn.metrics import classification_report
print(classification_report(results.Actual, results.Predicted))